In [1]:
from functools  import partial
import numpy as np

In [2]:
def e(t, v, i, a, L, d, h, t0 = -10):
    return (
        1/16*((2*L*i - 2*t*v - d + 2*h)*np.abs(-2*L*i + 2*t*v + d - 2*h) - 
          (2*L*i - 2*t*v - d)*np.abs(-2*L*i + 2*t*v + d) - 
          (2*L*i - 2*t*v + d + 2*h)*np.abs(-2*L*i + 2*t*v - d - 2*h) + 
          (2*L*i - 2*t*v + d)*np.abs(-2*L*i + 2*t*v - d) - 
          (2*L*i - 2*t0*v - d + 2*h)*np.abs(-2*L*i + 2*t0*v + d - 2*h) + 
          (2*L*i - 2*t0*v - d)*np.abs(-2*L*i + 2*t0*v + d) + 
          (2*L*i - 2*t0*v + d + 2*h)*np.abs(-2*L*i + 2*t0*v - d - 2*h) - 
          (2*L*i - 2*t0*v + d)*np.abs(-2*L*i + 2*t0*v - d))*a/h
    )

def t1(v, i, L, d):
    return (L*i+d/2)/v
def t2(v, i, L, d):
    t1_ = t1(v, i, L, d)
    return t1_ + h/v
def t3(v, i, L, d):
    t2_ = t2(v, i, L, d)
    return t2_ + (d-h)/v
def t4(v, i, L, d):
    t3_ = t3(v, i, L, d)
    return t3_ + h/v

In [3]:
def simulate_events(
    N = 100,
    threshold = 0.5,
    offset = 5,
    a=10, 
    L=0.1, 
    h=0.08, 
    d=0.2, 
    v=0.4, 
    t0=0,
    t1=100,
    t_step = 0.1
):

    current_em = np.zeros(N)
    total_em = np.zeros(N)
    events = np.zeros(N)
    eNs = []

    values = dict(a=a, L=L, h=h, d=d, v=v)
    e0 = partial(e, **values)


    for time in np.arange(t0,t1,t_step):
        pixel = np.arange(N)
        this_e = e0(t=time, i = pixel+1)
        total_em[pixel] = this_e
        eN = total_em[pixel] - current_em[pixel]

        event_mask = (eN > threshold)
        current_em[event_mask] = total_em[event_mask]
        events[event_mask] += 1
        
    return events

In [4]:
vels = np.arange(0,0.1,0.01)
events = np.array([simulate_events(v=v) for v in vels])
total_events = np.sum(events, axis = 1)


In [5]:
vels

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09])

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(vels, total_events)